In [1]:
import gym
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' 
import tensorflow as tf
import numpy as np

In [2]:
env = gym.make("CartPole-v0")

In [3]:
obs_dim = env.observation_space.shape[0]
n_acts = env.action_space.n
print("obs = {}, n_acts = {}".format(obs_dim, n_acts))

obs = 4, n_acts = 2


In [4]:
obs = env.reset()
obs

array([ 0.02363651,  0.02326473, -0.04545016, -0.01668425])

In [5]:
def mpl(x, sizes, activation = tf.tanh, output_activation=None):

    for size in sizes:
        x = tf.layers.dense(x,units = size, activation = activation)
    
    return tf.layers.dense(x, units = sizes[-1], activation = output_activation)


In [6]:
#first layer of NN
obs_ph = tf.placeholder(shape=(None, obs_dim), dtype=tf.float32)
#creating NN
logit = mpl(obs_ph, sizes = [32,64]+[n_acts])
#array([[0.01685937, 0.00622761]], dtype=float32)

Instructions for updating:
Use keras.layers.dense instead.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


In [7]:
max_action = tf.multinomial(logits=logit,num_samples=1)
#array([[0]])
action = tf.squeeze(max_action, axis=1)
#array([0])

Instructions for updating:
Use `tf.random.categorical` instead.


In [8]:
#loss
weights_ph = tf.placeholder(shape=(None,), dtype = tf.float32)
act_ph = tf.placeholder(shape=(None,), dtype = tf.int32)
#one_hot taken actions
action_masks = tf.one_hot(act_ph, n_acts)
#array([[0., 1.]...], dtype=float32)

#logprob(a|s)
log_probs = tf.reduce_sum(action_masks * tf.nn.log_softmax(logit), axis=1)
#array([-0.6952652,...], dtype=float32)
loss = -tf.reduce_mean(weights_ph* log_probs)
#9.812662

In [9]:
#optimizer
train_op = tf.train.AdamOptimizer(learning_rate = 0.01).minimize(loss)
#loss minimizes :D

In [10]:
#variables for initializing and using
sess = tf.InteractiveSession()
sess.run(tf.global_variables_initializer())

In [11]:
max_action = sess.run(action, {obs_ph: obs.reshape(1,-1)})[0]
#get element from array 0 or 1

In [12]:
def train_one_epoch(batch_size = 5000):
    
    batch_obs = []
    batch_acts = []
    batch_rets = []
    batch_lens = []
    batch_weights = []
    obs = env.reset()
    done = False
    ep_rews = []
    rendering_epoch = True
    
    while True:
        
        if rendering_epoch == True:
            pass
            #env.render() 
        batch_obs.append(obs.copy())
        
        act = sess.run(action, {obs_ph: obs.reshape(1,-1)})[0]
        
        obs, reward, done, _ = env.step(act)
        
        batch_acts.append(act)
        ep_rews.append(reward)
        
        if done:
            rendering_epoch = False
            #recording everything!
            ep_ret, ep_len = sum(ep_rews), len(ep_rews)
            
            batch_rets.append(ep_ret)
            batch_lens.append(ep_len)
            
            #weights for each logprob(a|s) is R(tau)
            batch_weights += [ep_ret] * ep_len
            #[ep_ret] - [16], *ep_len -> [16,16,16...16] 
            
            #reset vatiables
            obs, done, ep_rews = env.reset(), False, []
            
            if len(batch_obs)>batch_size:
                break
    
    if np.mean(batch_lens)<200:
        batch_loss, _ = sess.run([loss, train_op], {obs_ph: batch_obs,
                                             act_ph: batch_acts,
                                             weights_ph: batch_weights})
    else:
        batch_loss = 0
    
    return batch_loss, batch_rets, batch_lens


In [13]:
for i in range(500):
    batch_loss, batch_rets, batch_lens = train_one_epoch()
    print("#%i, batch_loss: %.3f, batch_rets: %.3f, batch_lens: %.3f" \
          %(i, batch_loss, np.mean(batch_rets), np.mean(batch_lens)))
    if batch_loss == 0:
        break


#0, batch_loss: 25.929, batch_rets: 30.404, batch_lens: 30.404
#1, batch_loss: 34.198, batch_rets: 42.432, batch_lens: 42.432
#2, batch_loss: 39.281, batch_rets: 53.925, batch_lens: 53.925
#3, batch_loss: 41.793, batch_rets: 58.953, batch_lens: 58.953
#4, batch_loss: 47.429, batch_rets: 70.873, batch_lens: 70.873
#5, batch_loss: 58.450, batch_rets: 87.190, batch_lens: 87.190
#6, batch_loss: 62.918, batch_rets: 97.346, batch_lens: 97.346
#7, batch_loss: 73.476, batch_rets: 117.651, batch_lens: 117.651
#8, batch_loss: 89.885, batch_rets: 149.118, batch_lens: 149.118
#9, batch_loss: 95.327, batch_rets: 174.931, batch_lens: 174.931
#10, batch_loss: 93.345, batch_rets: 173.069, batch_lens: 173.069
#11, batch_loss: 96.981, batch_rets: 176.966, batch_lens: 176.966
#12, batch_loss: 97.021, batch_rets: 185.444, batch_lens: 185.444
#13, batch_loss: 90.363, batch_rets: 170.800, batch_lens: 170.800
#14, batch_loss: 86.310, batch_rets: 164.806, batch_lens: 164.806
#15, batch_loss: 83.989, batch_ret

In [14]:
def play_episode():
    
    rewards = 0
    obs = env.reset()
    while True:
        env.render()
        act = sess.run(action, {obs_ph: obs.reshape(1,-1)})[0]
        obs, reward, is_done, _ = env.step(act)
        rewards += reward
        #if is_done or rewards >200:
            #break
    return rewards    

In [17]:
for i in range(50):
    print(play_episode())

KeyboardInterrupt: 

t = 2


x


y

t=2
while t!=100000000000000:
    x = t**(1/(t-1))
    y = t**(t/(t-1))
    if x.is_integer() and y.is_integer():
        print("x: {}^(1/{})   y: {}^{}/{}".format(t, t-1, t, t,t-1))
        print("t = {} x = {} y = {}".format(t, x, y))
    t+=1

4.0.is_integer()

t

In [ ]:
env.close()